# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Bring in cities data
city_weather_df = pd.read_csv("../WeatherPy/output_files/cities.csv")

#Display df
city_weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,23.00,50,99,1.01,US,1612199914
1,Alofi,-19.0595,-169.9187,75.20,100,90,4.61,NU,1612200185
2,Tuktoyaktuk,69.4541,-133.0374,-20.20,76,90,5.75,CA,1612200185
3,Buala,-8.1450,159.5921,81.10,78,100,13.87,SB,1612200185
4,Yellowknife,62.4560,-114.3525,-4.00,77,75,8.05,CA,1612199896
...,...,...,...,...,...,...,...,...,...
541,Port Blair,11.6667,92.7500,79.07,77,0,12.39,IN,1612200232
542,Kerman,30.2832,57.0788,42.80,19,0,4.61,IR,1612200232
543,Kidal,18.4411,1.4078,83.16,11,0,6.53,ML,1612200232
544,Manzhouli,49.6000,117.4333,-24.59,81,81,7.99,CN,1612200232


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
#Store the latitude/longitude in a variable and the humidity in a separate variable
coordinates = city_weather_df[["Latitude","Longitude"]]
humidity = city_weather_df["Humidity"]


# Start with base map
fig = gmaps.figure()


#Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Create dataframe with my ideal weather conditions
ideal_weather_df = city_weather_df.loc[(city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80) &
                                       (city_weather_df["Humidity"]<65) & (city_weather_df["Cloudiness"] < 10) &
                                       (city_weather_df["Wind Speed"] < 10)]
#Drop rows with null values
ideal_weather_df = ideal_weather_df.dropna(how = 'any')

#Display df
ideal_weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,São Filipe,14.8961,-24.4956,76.86,59,0,9.69,CV,1612200188
47,Mazarrón,37.5992,-1.3149,73.40,33,0,3.44,ES,1612200188
110,Alnāvar,15.4333,74.7333,73.09,36,0,6.96,IN,1612200066
307,Lima,-12.0432,-77.0282,77.00,61,0,4.61,PE,1612200102
316,Bucerías,20.7500,-105.3333,75.20,64,1,3.47,MX,1612200211
323,Ixtapa,20.7000,-105.2000,75.20,64,1,3.13,MX,1612200212
336,Makkah al Mukarramah,21.4267,39.8261,76.24,64,0,6.40,SA,1612200213
349,Tiznit Province,29.5833,-9.5000,74.79,35,0,5.03,MA,1612200214
369,Chancay,-11.5714,-77.2672,77.00,61,0,4.61,PE,1612200216
401,Kāmalāpuram,14.5833,78.6500,72.43,64,0,6.64,IN,1612200219


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store df into hotel variable
hotel_df = ideal_weather_df

#Add a hotel name column
hotel_df["Hotel Name"]=""


In [7]:
#Set empty list for hotel name
hotels = []

#Loop through and pull the lat, long for each city in the df
for x in range(len(hotel_df)):
    latitude = hotel_df.iloc[x,1]
    longitude = hotel_df.iloc[x,2]

    #Set base url for google places nearby search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Set parameters for the pull from google
    params = {
        "location": f'{latitude},{longitude}',
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    
    #Pull responses
    response = requests.get(base_url, params=params).json()
    
    #Pull hotels if one is listed and append to the empty hotel list
    try:
        hotels.append(response['results'][0]['name'])
    
    #If an error, put an empty space into the hotel column
    except (KeyError, IndexError):
        hotels.append("")

#Fill the hotel name column with the hotel names        
hotel_df["Hotel Name"] = hotels


#Display df
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
43,São Filipe,14.8961,-24.4956,76.86,59,0,9.69,CV,1612200188,Tortuga B&B
47,Mazarrón,37.5992,-1.3149,73.40,33,0,3.44,ES,1612200188,Apartamentos Oasis de las Palmeras
110,Alnāvar,15.4333,74.7333,73.09,36,0,6.96,IN,1612200066,Ashwin Hotel & Lodge
307,Lima,-12.0432,-77.0282,77.00,61,0,4.61,PE,1612200102,Gran Hotel Bolivar Lima
316,Bucerías,20.7500,-105.3333,75.20,64,1,3.47,MX,1612200211,Villa del Palmar Flamingos Beach Resort & Spa ...
323,Ixtapa,20.7000,-105.2000,75.20,64,1,3.13,MX,1612200212,Container Inn Puerto Vallarta
336,Makkah al Mukarramah,21.4267,39.8261,76.24,64,0,6.40,SA,1612200213,"Makkah Clock Royal Tower, A Fairmont Hotel"
349,Tiznit Province,29.5833,-9.5000,74.79,35,0,5.03,MA,1612200214,
369,Chancay,-11.5714,-77.2672,77.00,61,0,4.61,PE,1612200216,Hotel Madre Perla
401,Kāmalāpuram,14.5833,78.6500,72.43,64,0,6.64,IN,1612200219,Gaffar


In [8]:
#Remove rows without hotels listed (not asked for in assignment but I wanted to do this)

#Change "" to NaN
nan_value = float("NaN")
changed_hotel_df = hotel_df.replace("", nan_value)

#Drop rows with NaN
cleaned_hotel_df = changed_hotel_df.dropna(subset = ['Hotel Name'])

#Display df
cleaned_hotel_df



,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
43,São Filipe,14.8961,-24.4956,76.86,59,0,9.69,CV,1612200188,Tortuga B&B
47,Mazarrón,37.5992,-1.3149,73.40,33,0,3.44,ES,1612200188,Apartamentos Oasis de las Palmeras
110,Alnāvar,15.4333,74.7333,73.09,36,0,6.96,IN,1612200066,Ashwin Hotel & Lodge
307,Lima,-12.0432,-77.0282,77.00,61,0,4.61,PE,1612200102,Gran Hotel Bolivar Lima
316,Bucerías,20.7500,-105.3333,75.20,64,1,3.47,MX,1612200211,Villa del Palmar Flamingos Beach Resort & Spa ...
323,Ixtapa,20.7000,-105.2000,75.20,64,1,3.13,MX,1612200212,Container Inn Puerto Vallarta
336,Makkah al Mukarramah,21.4267,39.8261,76.24,64,0,6.40,SA,1612200213,"Makkah Clock Royal Tower, A Fairmont Hotel"
369,Chancay,-11.5714,-77.2672,77.00,61,0,4.61,PE,1612200216,Hotel Madre Perla
401,Kāmalāpuram,14.5833,78.6500,72.43,64,0,6.64,IN,1612200219,Gaffar
488,Ashti,19.3833,76.2333,70.70,24,0,8.81,IN,1612200227,Sree Sant Savta Mali


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]
locations = cleaned_hotel_df[["Latitude", "Longitude"]] 

In [16]:
# Add marker layer ontop of heat map to show hotels and info boxes
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))